In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = './2021-ml-100-marathon-final-exam/image_data/train/'
X = []
Y = []

for dai in os.listdir(path+'daisy/'):
    img = cv2.imread(path+'daisy/'+dai)
    X.append(img)
    Y.append(0)
    
for dan in os.listdir(path+'dandelion/'):
    img = cv2.imread(path+'dandelion/'+dan)
    X.append(img)
    Y.append(1)
    
for r in os.listdir(path+'rose/'):
    img = cv2.imread(path+'rose/'+r)
    X.append(img)
    Y.append(2)
    
for sun in os.listdir(path+'sunflower/'):
    img = cv2.imread(path+'sunflower/'+sun)
    X.append(img)
    Y.append(3)
    
for tu in os.listdir(path+'tulip/'):
    img = cv2.imread(path+'tulip/'+tu)
    X.append(img)
    Y.append(4)

In [3]:
for i, p in enumerate(X):
    X[i] = cv2.resize(p, (299, 299))
X = np.array(X)/255

In [4]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

del X
del Y

In [5]:
del img

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
import tensorflow.keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D

from tensorflow.keras.models import Model
from tensorflow.keras.applications.xception import Xception

In [8]:
num_classes = 5

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

# 建立 ImageDataGenerator，並指定我們要做資料增強的數值範圍
data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

x_train shape: (2258, 299, 299, 3)
2258 train samples
565 test samples


In [9]:
# 建立 ResNet 模型
b_model = Xception(input_shape = (299, 299, 3), classes=1000, weights="imagenet", input_tensor = None)
x = b_model.output
x = Dense(units = 256)(x)
x = BatchNormalization(axis = 1)(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)
output_layer = Dense(units = 5, activation = "softmax")(x)
model = Model(inputs = b_model.input, outputs = output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
______________________________________________________________________________________________

In [10]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor ="val_loss", patience = 5)

In [11]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr = 0.0001),
              metrics=['accuracy'])

data_generator.fit(x_train)

batch_size = 4
epochs = 200
history = model.fit_generator(data_generator.flow(x_train, y_train, batch_size=batch_size),
                              epochs=epochs,
                              steps_per_epoch=int(len(x_train)/32),
                              verbose=1,
                              validation_data=(x_test, y_test),
                              callbacks = [reduce_lr])

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
Train for 70 steps, validate on 565 samples
Epoch 1/200
70/70 [==============================] - 45s 645ms/step - loss: 1.5564 - accuracy: 0.3857 - val_loss: 1.5938 - val_accuracy: 0.5469
Epoch 2/200
70/70 [==============================] - 37s 524ms/step - loss: 1.4008 - accuracy: 0.4786 - val_loss: 1.5502 - val_accuracy: 0.7469
Epoch 3/200
70/70 [==============================] - 37s 525ms/step - loss: 1.1517 - accuracy: 0.5964 - val_loss: 1.4407 - val_accuracy: 0.8035
Epoch 4/200
70/70 [==============================] - 37s 528ms/step - loss: 0.9892 - accuracy: 0.6464 - val_loss: 1.2876 - val_accuracy: 0.8319
Epoch 5/200
70/70 [==============================] - 37s 527ms/step - loss: 1.0184 - accuracy: 0.6286 - val_loss: 1.1865 - val_accuracy: 0.7912
Epoch 6/200
70/70 [==============================] - 38s 537ms/step - loss: 0.8945 - accuracy: 0.7014 - val_loss: 0.9789 - val_accuracy

Epoch 55/200
70/70 [==============================] - 37s 530ms/step - loss: 0.3624 - accuracy: 0.8964 - val_loss: 0.2970 - val_accuracy: 0.9168
Epoch 56/200
70/70 [==============================] - 37s 530ms/step - loss: 0.3551 - accuracy: 0.9036 - val_loss: 0.2937 - val_accuracy: 0.9150
Epoch 57/200
70/70 [==============================] - 37s 530ms/step - loss: 0.3156 - accuracy: 0.9036 - val_loss: 0.2916 - val_accuracy: 0.9204
Epoch 58/200
70/70 [==============================] - 37s 530ms/step - loss: 0.3841 - accuracy: 0.8821 - val_loss: 0.2908 - val_accuracy: 0.9186
Epoch 59/200
70/70 [==============================] - 37s 530ms/step - loss: 0.4357 - accuracy: 0.8750 - val_loss: 0.2916 - val_accuracy: 0.9150
Epoch 60/200
70/70 [==============================] - 37s 529ms/step - loss: 0.4121 - accuracy: 0.8821 - val_loss: 0.2904 - val_accuracy: 0.9204
Epoch 61/200
70/70 [==============================] - 37s 530ms/step - loss: 0.4761 - accuracy: 0.8429 - val_loss: 0.2939 - val_ac

KeyboardInterrupt: 

In [12]:
#存model
model.save('transfer_learning_xception.h5')

In [14]:
result = pd.read_csv('./2021-ml-100-marathon-final-exam/sample_submission_new.csv')

In [17]:
path = './2021-ml-100-marathon-final-exam/image_data/test/'
test_X = []

for dai in os.listdir(path):
    img = cv2.imread(path+dai)
    test_X.append(img)
    
for i, p in enumerate(test_X):
    test_X[i] = cv2.resize(p, (299, 299))
test_X = np.array(test_X)/255

x_test = test_X.astype('float32')

In [21]:
result['Predicted'] = np.argmax(model.predict(x_test),axis=1)
result.to_csv("transfer_learning_xception.csv",header = ["I","ID", "Predicted"], index = False)